# The Gene Ontology Knowledge Graph

The Gene Ontology (GO) project is a collaborative effort to address the need for consistent descriptions of gene products across databases. 

The GO project has developed three structured ontologies that describe gene products in terms of their associated biological processes, cellular components and molecular functions in a species-independent manner.

The Gene Ontology project provides controlled vocabularies of defined terms representing gene product properties. These cover three domains: 

* __Cellular Component__, the parts of a cell or its extracellular environment

* __Molecular Function__, the elemental activities of a gene product at the molecular level, such as binding or catalysis

* __Biological Process__, operations or sets of molecular events with a defined beginning and end, pertinent to the functioning of integrated living units: cells, tissues, organs, and organisms.

So every term that falls into the category Cellular Component can be found in the data and it will have a defined degree of relatedness to this concept.

![alt text](http://www.emiliosanfilippo.it/wp-content/uploads/2011/11/Ontology.jpg "What is an ontology?")

On the website of the Gene Ontology Consortium different versions of the ontology can be downloaded. The file is available in XML:OWL format which is a a subformat of RDF designed for ontologies. The raw_file is about 200 mb big. It looks like this:

In [4]:
with open("go-plus.owl") as raw_file:
    for i in range(95):
        l=raw_file.next()
        if i>80:
            print l

    <!-- http://purl.obolibrary.org/obo/IAO_0100001 -->



    <owl:AnnotationProperty rdf:about="http://purl.obolibrary.org/obo/IAO_0100001">

        <rdfs:label rdf:datatype="http://www.w3.org/2001/XMLSchema#string">term replaced by</rdfs:label>

    </owl:AnnotationProperty>

    





    <!-- http://purl.obolibrary.org/obo/RO_0002161 -->



    <owl:AnnotationProperty rdf:about="http://purl.obolibrary.org/obo/RO_0002161">

        <obo:IAO_0000425 rdf:datatype="http://www.w3.org/2001/XMLSchema#string">Class: ?X DisjointWith: RO_0002162 some ?Y</obo:IAO_0000425>

        <oboInOwl:hasDbXref rdf:datatype="http://www.w3.org/2001/XMLSchema#string">RO:0002161</oboInOwl:hasDbXref>

        <oboInOwl:hasOBONamespace rdf:datatype="http://www.w3.org/2001/XMLSchema#string">external</oboInOwl:hasOBONamespace>



Since owl is an rdf format we can use the library _rdflib_ to process the Gene Ontology graph. There is an rdf query language called __SPARQL__ which we can use on this graph. It is related to SQL in terms of syntax. Queries are always defined in triples of subject, predicate, object. Where predicates can be of the type relation and subjects and objects of type concept.

In [6]:
import rdflib

In [12]:
g = rdflib.Graph()
g.parse("go-plus.owl")

<Graph identifier=N716e1fe6ee114fd4afb7f9937c2b025d (<class 'rdflib.graph.Graph'>)>

This very simple SPARQL query selects all the possible triplets of subject, predicate and object.

In [6]:
req = g.query("""
SELECT ?s WHERE { ?s ?p ?o }
""")

In [7]:
print len(req)

1786685


In [8]:
req=0

It is very slow and memory inefficient to store all triples in a list.

### Gene annotations
The gaf-file is a list of links between human proteins or genes and Gene Ontology concepts. 
>In GAF 2.1 format, the identifier must reference a top-level primary gene or gene product identifier: either a gene, or a protein that has a 1:1 correspondence to a gene.

In [13]:
from collections import defaultdict
annotations=defaultdict(list)
with open("goa_human.gaf") as anno_file:
    for line in anno_file:
        if not line.startswith("!"):
            d=line.split("\t")
            annotations[d[1]].append(d)

In [14]:
print len(annotations)


19486


In [15]:
import numpy as np
print "Average terms per gene/protein:",np.mean([len(ann) for key,ann in annotations.items()])
print 
print "First 3 annotations for the Uniprot protein Alpha-2-macroglobulin (P01023):\n---------------------"
print "\n\n".join([", ".join(l) for l in annotations["P01023"][:3]])

Average terms per gene/protein: 22.5355639947

First 3 annotations for the Uniprot protein Alpha-2-macroglobulin (P01023):
---------------------
UniProtKB, P01023, A2M, , GO:0001869, PMID:12538697, IDA, , P, Alpha-2-macroglobulin, A2MG_HUMAN|A2M|CPAMD5|FWP007, protein, taxon:9606, 20090415, UniProt, , 


UniProtKB, P01023, A2M, , GO:0002020, PMID:18485748, IPI, UniProtKB:P58397, F, Alpha-2-macroglobulin, A2MG_HUMAN|A2M|CPAMD5|FWP007, protein, taxon:9606, 20131206, BHF-UCL, , 


UniProtKB, P01023, A2M, , GO:0002020, PMID:18485748, IPI, UniProtKB:Q9UKP4, F, Alpha-2-macroglobulin, A2MG_HUMAN|A2M|CPAMD5|FWP007, protein, taxon:9606, 20131206, BHF-UCL, , 



In [16]:
req2= g.query("""
    SELECT DISTINCT ?s WHERE { ?s ?p ?o } LIMIT 10
    """)

In [17]:
for s in req2:
    print s[0]

Na5aac9bfb83f48f0a92c504f81967f25
http://purl.obolibrary.org/obo/GO_0070965
http://purl.obolibrary.org/obo/GO_0008379
N43e18c50be2e45e9abc9ba81da7c8e12
Nac559f96344843148ae88af3d77d8c8e
Na68c6c88a91e4770b2215855b3f1e9d8
http://purl.obolibrary.org/obo/CHEBI_25418
http://purl.obolibrary.org/obo/GO_0036032
N8f61bc9a0ac54df3a1a62076d7189ddb
N861c2ebbfe4b49419297e7ae5d7fbde0


In [18]:
def get_description(GO_id):
    my_id=GO_id.replace(":","_")
    desc="http://purl.obolibrary.org/obo/IAO_0000115"
    req= g.query("""
    SELECT ?description WHERE
    {{<http://purl.obolibrary.org/obo/{}> <{}> ?description}}
    """.format(my_id,desc))
    return [d for d in req][0][0]

In [19]:
def get_label(GO_id):
    my_id=GO_id.replace(":","_")
    req= g.query("""
    SELECT ?description WHERE
    {{<http://purl.obolibrary.org/obo/{}> <http://www.w3.org/2000/01/rdf-schema#label> ?description}}
    """.format(my_id))
    return [d for d in req][0][0]

In [20]:
print get_label("GO:0006351")
print get_description("GO:0006351")

transcription, DNA-templated
The cellular synthesis of RNA on a template of DNA.


In [21]:
def get_subclasses(GO_id):
    my_id=GO_id.replace(":","_")
    req= g.query("""
    SELECT ?class WHERE
    {{
    ?class <http://www.w3.org/2000/01/rdf-schema#subClassOf> <http://purl.obolibrary.org/obo/{}>.
    }}
    """.format(my_id))
    return [str(d[0]) for d in req if type(d[0])==rdflib.term.URIRef]

In [22]:
def get_superclasses(GO_id):
    my_id=GO_id.replace(":","_")
    req= g.query("""
    SELECT ?class WHERE
    {{
     <http://purl.obolibrary.org/obo/{}> <http://www.w3.org/2000/01/rdf-schema#subClassOf> ?class.
    }}
    """.format(my_id))
    return [str(d[0]) for d in req if type(d[0])==rdflib.term.URIRef]

In [23]:
for sub in get_subclasses("GO:0006351"):
    print sub

http://purl.obolibrary.org/obo/GO_0098781
http://purl.obolibrary.org/obo/GO_0006360
http://purl.obolibrary.org/obo/GO_0009299
http://purl.obolibrary.org/obo/GO_0006390
http://purl.obolibrary.org/obo/GO_0009300
http://purl.obolibrary.org/obo/GO_0001059
http://purl.obolibrary.org/obo/GO_0097393
http://purl.obolibrary.org/obo/GO_0042793
http://purl.obolibrary.org/obo/GO_0006383
http://purl.obolibrary.org/obo/GO_0006366
http://purl.obolibrary.org/obo/GO_0001121
http://purl.obolibrary.org/obo/GO_0001060


In [24]:
for sup in get_superclasses("GO:0006351"):
    print sup

http://purl.obolibrary.org/obo/GO_0034645
http://purl.obolibrary.org/obo/GO_0097659


In [25]:
import networkx as nx
import itertools

def create_close_graph(GO_id,sub_vic,sup_vic):
    G = nx.Graph()
    my_id="http://purl.obolibrary.org/obo/"+GO_id.replace(":","_")
    max_lev=sub_vic+sup_vic+1
    max_w=0
    G.add_node(my_id,level=0,order=0,pos=[0.5,(sub_vic+1)*(1.0/max_lev)],info=get_label(GO_id)+"<br>"+get_description(GO_id))
    temp_sub_classes=[my_id]
    for i in range(sub_vic):
        new_classes=[]
        for cl in temp_sub_classes:
            sub=get_subclasses(cl.split("/")[-1])
            new_classes.append((cl,sub))
        
        new_classes_all=list(itertools.chain(*[s for c,s in new_classes]))
        max_w=len(new_classes_all)
        ind=0
        for c,sub_c in new_classes:
            for k,s in enumerate(sub_c):
                ind+=1
                G.add_node(s,level=(i+1),pos=(ind*(1.0/(max_w+1)),(sub_vic-i)*(1.0/max_lev)),info=get_label(s.split("/")[-1]))
                G.add_edge(c,s)
                
        temp_sub_classes=new_classes_all
    

    temp_sup_classes=[my_id]
    for i in range(sup_vic):
        new_classes=[]
        for cl in temp_sup_classes:
            sup=get_superclasses(cl.split("/")[-1])
            new_classes.append((cl,sup))
        
        new_classes_all=list(itertools.chain(*[s for c,s in new_classes]))
        max_w=len(new_classes_all)
        ind=0
        for c,sup_c in new_classes:
            for k,s in enumerate(sup_c):
                ind+=1
                G.add_node(s,level=(i+1)*-1,pos=(ind*(1.0/(max_w+1)),(i+sub_vic+2)*(1.0/max_lev)),info=get_label(s.split("/")[-1]))
                G.add_edge(c,s)
                
        temp_sup_classes=new_classes_all
        
    return G

In [26]:
graph=create_close_graph("GO:0006351",sup_vic=3,sub_vic=1)

In [27]:
graph.nodes()

['http://purl.obolibrary.org/obo/GO_0001059',
 'http://purl.obolibrary.org/obo/GO_0009058',
 'http://purl.obolibrary.org/obo/GO_0009059',
 'http://purl.obolibrary.org/obo/GO_1901576',
 'http://purl.obolibrary.org/obo/GO_0016070',
 'http://purl.obolibrary.org/obo/GO_0001121',
 'http://purl.obolibrary.org/obo/GO_0006360',
 'http://purl.obolibrary.org/obo/GO_0006366',
 'http://purl.obolibrary.org/obo/GO_0097393',
 'http://purl.obolibrary.org/obo/GO_0034654',
 'http://purl.obolibrary.org/obo/GO_0044237',
 'http://purl.obolibrary.org/obo/GO_0006390',
 'http://purl.obolibrary.org/obo/GO_0009299',
 'http://purl.obolibrary.org/obo/GO_0009300',
 'http://purl.obolibrary.org/obo/GO_0097659',
 'http://purl.obolibrary.org/obo/GO_0001060',
 'http://purl.obolibrary.org/obo/GO_0042793',
 'http://purl.obolibrary.org/obo/GO_0098781',
 'http://purl.obolibrary.org/obo/GO_0006351',
 'http://purl.obolibrary.org/obo/GO_0034645',
 'http://purl.obolibrary.org/obo/GO_0044260',
 'http://purl.obolibrary.org/obo/G

In [28]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True)

In [29]:
def visualize_go_term(G):
    edge_trace = Scatter(
        x=[],
        y=[],
        line=Line(width=0.5,color='#888'),
        hoverinfo='none',
        mode='lines')

    for edge in G.edges():
        x0, y0 = G.node[edge[0]]['pos']
        x1, y1 = G.node[edge[1]]['pos']
        edge_trace['x'] += [x0, x1, None]
        edge_trace['y'] += [y0, y1, None]

    node_trace = Scatter(
        x=[],
        y=[],
        text=[],
        mode='markers',
        hoverinfo='text',
        marker=Marker(
        colorscale='Viridis',
        reversescale=True,
        color=[],
        size=10,
        line=dict(width=2)))
        
    
    for node in G.nodes():
        x, y = G.node[node]['pos']
        node_trace['marker']['color'].append(G.node[node]['level'])
        node_trace['text'].append(G.node[node]['info'])
        node_trace['x'].append(x)
        node_trace['y'].append(y)
    
        
    fig = Figure(data=Data([edge_trace, node_trace]),
             layout=Layout(
                title='<br>Network of related terms in Gene Ontology',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=50),
                annotations=[ ],
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))

    iplot(fig, filename='networkx')

In [30]:
visualize_go_term(graph)

In [31]:
def inspect_annotations(anns,UniProt_id=None):
    if not UniProt_id:
        gene_id=np.random.choice(anns.keys())
    else:
        gene_id=UniProt_id
    rel_terms=[a for a in annotations[gene_id]]
    
    print "found",len(rel_terms)," GO annotations for term",rel_terms[0][9],"("+rel_terms[0][1]+")!"
    print "graph representations produced!",
    rel_graphs=[create_close_graph(term[4],sup_vic=2,sub_vic=1) for term in rel_terms]
    return iter(rel_graphs)

In [32]:
graphs=inspect_annotations(annotations,"A6NMS3")

found 8  GO annotations for term Olfactory receptor 5K4 (A6NMS3)!
graph representations produced!


In [33]:
try:
    visualize_go_term(graphs.next())
except StopIteration:
    print "end of list."